保险场景 VLM使用

In [ ]:
import os
from openai import OpenAI
import pandas as pd

client = OpenAI(
    api_key="your_api_key_here", 
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

# 调用VLM，得到推理结果
# user_prompt：用户想要分析的内容
# image_url：想要分析的图片
def get_response(user_prompt, image_url):
    # 得到image_url_list，一张图片也放到[]中
    if image_url.startswith('[') and ',' in image_url:
        # 判断 image_url 是否为列表格式，若以 '[' 开头且包含 ','，则认为传入的是一个图片 URL 列表
        # 去除 image_url 字符串两端的空白字符
        image_url = image_url.strip()
        # 去除字符串首尾的 '[' 和 ']'，以获取实际的 URL 内容
        image_url = image_url[1:-1]
        # 以逗号为分隔符，将字符串分割成多个 URL 组成的列表
        image_url_list = image_url.split(',')
        # 对分割后的每个 URL 去除两端的空白字符，保证 URL 的纯净性
        image_url_list = [temp_url.strip() for temp_url in image_url_list]
    else:
        image_url_list = [image_url]
    # 得到messages
    content = [{"type": "text", "text": f"{user_prompt}"}]
    for temp_url in image_url_list: 
        image_url = f"https://vl-image.oss-cn-shanghai.aliyuncs.com/{temp_url}.jpg"
        content.append({"type": "image_url", "image_url": {"url": image_url}})
    messages = [
        {
            "role": "user",
            "content": content
        }
    ]
    # JSON 样例格式
    """
    {
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "请分析这些图片"
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": "https://vl-image.oss-cn-shanghai.aliyuncs.com/image1.jpg"
                        }
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": "https://vl-image.oss-cn-shanghai.aliyuncs.com/image2.jpg"
                        }
                    }
                ]
            }
        ]
    }
    """

    print(f'messages={messages}')
    completion = client.chat.completions.create(
        model="qwen-vl-max", #qwen-vl-plus
        messages=messages    
        )
    print(completion.model_dump_json())
    return completion

In [ ]:
%pip install openpyxl

In [ ]:
# 从当前目录下读取名为 'prompt_template_cn.xlsx' 的 Excel 文件，并将其内容存储到 DataFrame 中
df = pd.read_excel('./prompt_template_cn.xlsx')
# 在 DataFrame 中新增一列 'response'，并将初始值都设为空字符串
df['response'] = ''

# 遍历 DataFrame 的每一行
for index, row in df.iterrows():
    # 从当前行中获取提示信息
    user_prompt = row['prompt']
    # 从当前行中获取图片的 URL
    image_url = row['image']
    
    # 调用 get_response 函数，传入提示信息和图片 URL，获取 VLM 模型的推理结果
    completion = get_response(user_prompt, image_url)
    # 从推理结果中提取第一条消息的内容
    response = completion.choices[0].message.content
    # 将提取的响应内容填充到当前行的 'response' 列中
    df.loc[index, 'response'] = response
    # 打印当前处理的行号、提示信息和图片 URL
    print(f"{index+1} {user_prompt} {image_url}")

# 将处理后的 DataFrame 保存为 Excel 文件，文件名为 'prompt_template_cn_result.xlsx'，且不保存行索引
df.to_excel('./prompt_template_cn_result.xlsx', index=False)

In [ ]:
df